# Deep Galerkin Method (DGM)
### S1 = sigma(w1*x + b1)  Z(l) = sigma(u*x + w*S + b) l=1,...,L  G(l) = sigma(u*x + w*S + b) l=1,...,L  
### R(l) = sigma(u*x + w*S + b) l=1,...,L   H(l) = sigma(u*x + w*(S Hadamard R) + b)  l=1,...,L  
### S(L+1) = (1-G) Hadamard H + Z Hadamard S  f = w*S(L+1) + b  

In [1]:
#import needed packages
import tensorflow as tf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

/home/wangyaohong/.conda/envs/tensorflow1.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wangyaohong/.conda/envs/tensorflow1.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wangyaohong/.conda/envs/tensorflow1.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wangyaohong/.

**input_dim:** dimensionality of imput data  
**output_dim:** number of output for LSTM layer  
**trans1, trans2 (str):** activation functions used inside the layer;  
one of: "tanh"(default), "relu" or "sigmoid"  
**u vectors:** weighting vectors for inputs original inputs x  
**w vectors:** weighting vectors for output of previous layer  
**S:** output of previous layer  
**X:** data input

In [2]:
# LSTM-like layer used in DGM
class LSTMLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, trans1 = "tanh", trans2 = "tanh"):
        
        super(LSTMLayer, self).__init__()
        
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        if trans1 == "tanh":
            self.trans1 = tf.nn.tanh
        elif trans1 == "relu":
            self.trans1 = tf.nn.relu
        elif trans1 == "sigmoid":
            self.trans1 = tf.nn.sigmoid
        
        if trans2 == "tanh":
            self.trans2 = tf.nn.tanh
        elif trans2 == "relu":
            self.trans2 = tf.nn.relu
        elif trans2 == "sigmoid":
            self.trans2 = tf.nn.relu
        
        # u vectors (weighting vectors for inputs original inputs x)
        self.Uz = self.add_variable("Uz", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Ug = self.add_variable("Ug", shape=[self.input_dim ,self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Ur = self.add_variable("Ur", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Uh = self.add_variable("Uh", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        
        # w vectors (weighting vectors for output of previous layer)        
        self.Wz = self.add_variable("Wz", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wg = self.add_variable("Wg", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wr = self.add_variable("Wr", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        self.Wh = self.add_variable("Wh", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.contrib.layers.xavier_initializer())
        
        # bias vectors
        self.bz = self.add_variable("bz", shape=[1, self.output_dim])
        self.bg = self.add_variable("bg", shape=[1, self.output_dim])
        self.br = self.add_variable("br", shape=[1, self.output_dim])
        self.bh = self.add_variable("bh", shape=[1, self.output_dim])
    
    
    def call(self, S, X):

        Z = self.trans1(tf.add(tf.add(tf.matmul(X,self.Uz), tf.matmul(S,self.Wz)), self.bz))
        G = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ug), tf.matmul(S, self.Wg)), self.bg))
        R = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ur), tf.matmul(S, self.Wr)), self.br))
        
        H = self.trans2(tf.add(tf.add(tf.matmul(X,self.Uh), tf.matmul(tf.multiply(S, R), self.Wh)), self.bh))
        
        S_new = tf.add(tf.multiply(tf.subtract(tf.ones_like(G), G), H), tf.multiply(Z,S))
        
        return S_new

**input_dim:** dimensionality of input data  
**output_dim:** number of outputs for dense layer  
**transformation:** activation function used inside the layer; using None is equivalent to the identity map  
**w vectors:** weighting vectors for output of previous layer  
**X:** input to layer  

In [3]:
# Fully connected layer(dense)
class DenseLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, transformation=None):
        
        super(DenseLayer,self).__init__()
        self.output_dim = output_dim
        self.input_dim = input_dim

        self.W = self.add_variable("W", shape=[self.input_dim, self.output_dim],
                                   initializer = tf.contrib.layers.xavier_initializer())
        
        # bias vectors
        self.b = self.add_variable("b", shape=[1, self.output_dim])
        
        if transformation:
            if transformation == "tanh":
                self.transformation = tf.tanh
            elif transformation == "relu":
                self.transformation = tf.nn.relu
        else:
            self.transformation = transformation
    
    
    def call(self,X):
        
        S = tf.add(tf.matmul(X, self.W), self.b)
                
        if self.transformation:
            S = self.transformation(S)
        
        return S

**n_layers:** number of intermediate LSTM layers  
**input_dim:** spaital dimension of input data (excludes time dimension)  
**final_trans:** transformation used in final layer
define initial layer as fully connected
to account for time inputs we use input_dim+1 as the input dimension
**t:** sampled time inputs  
**x:** sampled space inputs  
Run the DGM model and obtain fitted function value at the inputs (t,x)

In [4]:
# Neural network architecture used in DGM

class DGMNet(tf.keras.Model):
    
    def __init__(self, layer_width, n_layers, input_dim, final_trans=None):
        
        super(DGMNet,self).__init__()
        
        self.initial_layer = DenseLayer(layer_width, input_dim+1, transformation = "tanh")
        
        self.n_layers = n_layers
        self.LSTMLayerList = []
                
        for _ in range(self.n_layers):
            self.LSTMLayerList.append(LSTMLayer(layer_width, input_dim+1))
        
        self.final_layer = DenseLayer(1, layer_width, transformation = final_trans)
    
    def call(self,t,x):

        X = tf.concat([t,x],1)
        
        # call initial layer
        S = self.initial_layer.call(X)
        
        # call intermediate LSTM layers
        for i in range(self.n_layers):
            S = self.LSTMLayerList[i].call(S,X)
        
        # call final LSTM layers
        result = self.final_layer.call(S)
        
        return result

Parameters

In [5]:
# OU process parameters（Ornstein-Uhlenbeck Process）
kappa = 0
theta = 0.5
sigma = 2

# mean and standard deviation for (normally distributed) process starting value
alpha = 0.0
beta = 1

# tenminal time
T = 1.0

# bounds of sampling region for space dimension, i.e. sampling will be done on
# [multipliter*Xlow, multiplier*Xhigh]
Xlow = -4.0
Xhigh = 4.0
x_multiplier = 2.0
t_multiplier = 1.5

# neural network parameters
num_layers = 3
nodes_per_layer = 50
learning_rate = 0.001

# Training parameters
sampling_stages = 500
steps_per_sample = 10

# Sampling parameters
nSim_t = 5
nSim_x_interior = 50
nSim_x_initial = 50

# Save options
saveName = 'FokkerPlanck'
saveFigure = False

**OU Simulation function(Ornstein-Uhlenbeck Process)**  
Simulate end point of Ornstein-Uhlenbeck process with normally distributed random starting value  
**alpha:** mean of random starting value  
**beta:** standard deviation of random starting value   
**theta:** mean reversion level    
**kappa:** mean reversion rate  
**sigma:** volatility  
**nSim:** number of simulations  
**T:** terminal time  

In [6]:
def simulateOU_GaussianStart(alpha, beta, theta, kappa, sigma, nSim, T):
    
    # simulate initial point based on normal distribution
    X0 = np.random.normal(loc = alpha, scale = beta, size = nSim)
    
    # mean and variance of OU endpoint
    m = theta + (X0 - theta) * np.exp(-kappa * T)
    v = np.sqrt(sigma**2 / (2 * kappa) * (1 - np.exp(-2*kappa*T)))
    
    # simulate endpoint
    Xt = np.random.normal(m,v)    
    
    return Xt

Sample time-space points from the function's domain;  
point are sampled uniformly on the interior of the domain, at the initial/terminal time points  
and along the spatial boundary at different time points.  
**nSim_t:** number of (interior) time points to sample  
**nSim_x_interior:** number of space points in the interior of the function's domain to sample  
**nSim_x_initial:** number of space points at initial time to sample (initial condition)

In [7]:
# Sampling function - random sample time-space pairs
def sampler(nSim_t, nSim_x_interior, nSim_x_initial):
    
    # Sampler1: domain interior
    t = np.random.uniform(low=0, high=T*t_multiplier, size=[nSim_t, 1])
    x_interior = np.random.uniform(low=Xlow*x_multiplier, high=Xhigh*x_multiplier, size=[nSim_x_interior, 1])
    
    # Sampler: spatial boundary
    # no spatial boundary condition for this problem 
    
    # Sampler3: initial/terminal condition
    x_initial = np.random.uniform(low=Xlow*1.5, high=Xhigh*1.5, size = [nSim_x_initial, 1])
    
    return t, x_interior, x_initial

Compute total loss for training.     
The loss is based o the PDE satisfied by the negative-exponential of the density and NOT the density   
itself, i.e. the u(t,x) in p(t,x) = exp(-u(t,x)) / c(t) where p is the density and c is the normalization constant.    
**model:** DGM model object   
**t:** sampled (interior) time points  
**x_interior:** sampled space points in the interior of the function's domain   
**x_initial:** sampled space points at initial time   
**nSim_t:** number of (interior) time points sampled (size of t)  
**alpha:** mean of normal distribution for process staring value  
**beta:** standard deviation of normal distribution for process starting value  


In [8]:
# Loss function for Fokker-Planck equation

def loss(model, t, x_interior, x_initial, nSim_t, alpha, beta):
    
    # Loss term1: PDE
    
    # initialize vector of losses
    losses_u = []
    
    # for each simulated interior time point
    for tIndex in range(nSim_t):
        
        curr_t = t[tIndex]
        t_vector = curr_t * tf.ones_like(x_interior)
        
        u    = model.call(t_vector, x_interior)
        u_t  = tf.gradients(u, t_vector)[0]
        u_x  = tf.gradients(u, x_interior)[0]
        u_xx = tf.gradients(u_x, x_interior)[0]

        psi_denominator = tf.reduce_sum(tf.exp(-u))
        psi = tf.reduce_sum( u_t*tf.exp(-u) ) / psi_denominator

        # PDE differential operator
        diff_f = -u_t + kappa - kappa*(x_interior- theta)*u_x - 0.5*sigma**2*(-u_xx + u_x**2) + psi
        
        # compute L2-norm of differential operator and attach to vector of losses
        currLoss = tf.reduce_mean(tf.square(diff_f)) 
        losses_u.append(currLoss)
    
    # average losses across sample time points 
    L1 = tf.add_n(losses_u) / nSim_t
    
    # Loss term2: boundary condition
    # no boundary condition for this problem
    
    # Loss term3: initial condition
    # compute negative-exponential of neural network-implied pdf at t = 0 i.e. the u in p = e^[-u(t,x)] / c(t)
    fitted_pdf = model.call(0*tf.ones_like(x_initial), x_initial)
    
    target_pdf  = 0.5*(x_initial - alpha)**2 / (beta**2)
    
    # average L2 error for initial distribution
    L3 = tf.reduce_mean(tf.square(fitted_pdf - target_pdf))

    return L1, L3

##### input(time, space domain interior, space domain at initial time)

In [9]:
# Set up network
model = DGMNet(nodes_per_layer, num_layers, 1)

t_tnsr = tf.placeholder(tf.float32, [None,1])
x_interior_tnsr = tf.placeholder(tf.float32, [None,1])
x_initial_tnsr = tf.placeholder(tf.float32, [None,1])

# loss 
L1_tnsr, L3_tnsr = loss(model, t_tnsr, x_interior_tnsr, x_initial_tnsr, nSim_t, alpha, beta)
loss_tnsr = L1_tnsr + L3_tnsr

u = model.call(t_tnsr, x_interior_tnsr)
p_unnorm = tf.exp(-u)

# set optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_tnsr)

# initialize variables
init_op = tf.global_variables_initializer()

# open session
sess = tf.Session()
sess.run(init_op)

Instructions for updating:
Use the retry module or similar alternatives.


In [10]:
#Train network
for i in range(sampling_stages):
    
    # sample uniformly from the required regions
    t, x_interior, x_initial = sampler(nSim_t, nSim_x_interior, nSim_x_initial)
    
    for j in range(steps_per_sample):
        loss,L1,L3,_ = sess.run([loss_tnsr, L1_tnsr, L3_tnsr, optimizer],
                                feed_dict = {t_tnsr:t, x_interior_tnsr:x_interior, x_initial_tnsr:x_initial})
        
    print(loss, L1, L3, i)

9.964574 1.9752108 7.989363 0
7.558194 2.5371182 5.021076 1
6.140235 1.9708465 4.1693883 2
2.6498854 0.35510942 2.294776 3
4.247659 1.5360676 2.7115917 4
2.940885 0.7515363 2.1893487 5
1.2956846 0.32190165 0.97378296 6
4.532461 1.6162363 2.916225 7
3.4336998 0.9740605 2.4596393 8
1.8899381 0.63668245 1.2532556 9
1.421838 0.29477286 1.1270652 10
2.6204557 1.5101446 1.1103112 11
2.302884 1.2689656 1.0339186 12
1.2147251 0.4158629 0.7988622 13
0.42135438 0.16873729 0.2526171 14
0.46387473 0.1352252 0.32864952 15
3.347396 2.161321 1.186075 16
1.2096891 0.47464666 0.7350425 17
0.5784742 0.26971483 0.3087594 18
0.5177029 0.1217124 0.3959905 19
0.5338627 0.323698 0.21016468 20
0.3608293 0.15026066 0.21056862 21
0.22447376 0.104209594 0.120264165 22
0.11324502 0.022282617 0.0909624 23
0.07513886 0.024736999 0.050401863 24
0.7040769 0.32685435 0.3772225 25
0.27144504 0.056406666 0.21503837 26
0.2845407 0.20239297 0.08214776 27
0.13123968 0.08004018 0.051199503 28
0.09274319 0.028800456 0.063942

0.3952098 0.2592 0.1360098 219
0.095554054 0.05615523 0.03939882 220
0.12713535 0.0877558 0.039379552 221
0.042312276 0.028957838 0.013354437 222
0.018804459 0.007921945 0.010882514 223
0.0526113 0.036474776 0.016136521 224
0.019459233 0.006611896 0.012847337 225
0.088387676 0.0743924 0.013995277 226
0.037566934 0.026237205 0.011329728 227
0.033574544 0.014671594 0.018902952 228
0.012020422 0.006501447 0.0055189747 229
0.057590246 0.047848094 0.009742153 230
0.047537927 0.035218336 0.012319592 231
0.0358931 0.032557543 0.0033355572 232
0.030280977 0.018624753 0.011656225 233
0.016398013 0.010526792 0.00587122 234
0.0690191 0.058154594 0.010864506 235
0.06748232 0.04723693 0.020245388 236
0.05630513 0.03108415 0.025220979 237
0.025405817 0.013770938 0.011634879 238
1.0990931 0.66152114 0.43757194 239
0.26982474 0.17359026 0.09623449 240
1.5879974 0.9678168 0.6201806 241
0.8178815 0.22363822 0.5942433 242
0.1881536 0.055411465 0.13274214 243
0.089475796 0.02185307 0.06762272 244
0.205415

0.041137993 0.034935128 0.0062028654 429
0.0056737885 0.0035497795 0.0021240092 430
0.031928323 0.02561959 0.0063087307 431
0.0075649535 0.0058937436 0.0016712099 432
0.06278678 0.03909015 0.023696633 433
0.029082328 0.013456878 0.01562545 434
0.060715146 0.03240586 0.028309284 435
0.0456945 0.03557747 0.010117032 436
0.014492616 0.009146318 0.005346298 437
0.0064986474 0.0031657224 0.003332925 438
0.004741301 0.003754323 0.000986978 439
0.00853553 0.0077443453 0.00079118507 440
0.012188656 0.010391766 0.0017968899 441
0.0030435165 0.0012014344 0.001842082 442
0.13523147 0.10774424 0.027487233 443
0.025954753 0.010736883 0.015217871 444
0.012495943 0.0058275037 0.0066684396 445
0.0076598637 0.0034328622 0.004227001 446
0.004415671 0.0013823673 0.0030333034 447
0.0049046115 0.0038963032 0.0010083084 448
0.09872846 0.07656217 0.022166286 449
0.018305298 0.010794403 0.0075108944 450
0.101342075 0.083813675 0.017528398 451
0.19511795 0.16396517 0.031152783 452
0.052327953 0.023906868 0.028

In [11]:
saver = tf.train.Saver()
saver.save(sess, './FokkerPlack/' + saveName)

'./FokkerPlack/FokkerPlanck'